In [32]:
import numpy as np
from numpy import cumsum

num_dogs = 30
num_trails = 25
y = np.loadtxt('data.txt')
#flipping the data
y_failure = 1 - y
data=y

#cumulative sum of all previous avoidances
xa = cumsum(y,axis=1) - y

#cumulative sum of all previous shocks
xs = cumsum(y_failure,axis=1) - y_failure


In [33]:
from scipy.stats import expon
from scipy.stats import norm

def likelihood(alpha,beta,data):
    p_log = np.zeros((30, 25), dtype=np.float64)
    p = np.zeros((30, 25), dtype=np.float64)
    p_log = alpha * xa + beta * xs
    p = np.exp(p_log)
   
    prob = np.zeros((30, 25), dtype=np.float64)
    for d in range(30):
        for t in range(25):
            #Bernoulli
            if data[d][t] == 0: 
                #storing log probability values
                prob[d][t] = math.log(p[d][t])
            else:
                #storing log probability values
                prob[d][t] = math.log(1 - p[d][t])

    likelihood = prob.sum()
    #returns log likelihood
    return likelihood

def prior(alpha,beta):
    alpha_prior = norm.pdf(alpha)
    beta_prior = norm.pdf(beta)
    
    return alpha_prior*beta_prior

def compute_posterior(alpha,beta, data):
    
    posterior = likelihood(alpha,beta,data) * prior(alpha,beta)
   
    return posterior


In [50]:
import numpy as np
import scipy.stats as stats
import scipy.optimize as optimize
import math

def ll_expectation(m1, v1, m2, v2):
        #sampling only negative values for alpha from a normal distribution with mean m1 and variance v1
        while True:
            #alpha = stats.norm.rvs(loc=m1, scale=math.sqrt(v1))
            alpha = stats.norm.rvs(loc=m1, scale=(v1))
            if alpha < -0.00001:
                break
        #sampling only negative values for beta from a normal distribution with mean m2 and variance v2
        while True:
            #beta = stats.norm.rvs(loc=m2, scale=math.sqrt(v2))
            beta = stats.norm.rvs(loc=m2, scale=(v2))
            if beta < -0.00001:
                break
           
        #finding log likelihood with the sampled values of alpha and beta
        ll = likelihood(alpha,beta,data)

        #prob_alpha = stats.norm.pdf(alpha, loc=m1, scale=math.sqrt(v1))
        #prob_beta = stats.norm.pdf(beta, loc=m2, scale=math.sqrt(v2))
        
        #finding probability of the sampled alpha and beta
        prob_alpha = stats.norm.pdf(alpha, loc=m1, scale=(v1))
        prob_beta = stats.norm.pdf(beta, loc=m2, scale=(v2))
      
        return ll * prob_alpha * prob_beta
    
def gauss_param(m1, v1, m2, v2):
        #finding mean and variance for the normal distribution which is a product of 2 other normal distributions with 
        #parameters m1,v1 and m2,v2 respectively
        m = (v1**2 * m2 + v2**2 * m1) / (v1**2 + v2**2)
        v = 1./((1./v1**2) + (1./v2**2))
        
        return m, v

def LowerBound(params):
        #free parameters generated by optimiser
        m1, v1, m2, v2 = params
             
        #findind mean and variance of approx. distribution 
        (approx_m, approx_v) = gauss_param(m1, v1, m2, v2)
        (prior_m, prior_v) = gauss_param(0,1,0,1)
        #finding kl divergene between approx and prior distributions
        kl = math.log(prior_v/approx_v) + (float(approx_v**2 + (approx_m - prior_m)**2)/(2 * prior_v**2)) - (1./2)
        
        #finding lower bound 
        lb = -kl + ll_expectation(m1, v1, m2, v2)
        #returning negative of lower bound to maximize
        return -lb

def optimiser():
        #initial parameters
        m1 = -0.25
        v1 = 1
        m2 = -0.08
        v2 = 1
        for i in range(0,1000):
            #passing lower bound function handler and the parameters to the optimization routine
            result = optimize.minimize(fun=LowerBound,x0=[m1, v1, m2, v2])
            m1, v1, m2, v2 = result["x"]
            print result["x"]
        m1, v1, m2, v2 = result["x"]
        print 'Alpha : Mean = ',m1,',Variance = ',v1
        print 'Beta :  Mean =',m2,',Variance = ' ,v2

In [ ]:
optimiser()

[-0.25  1.   -0.08  1.  ]
[-0.25000085  0.99999922 -0.08000059  0.99999938]
[-0.25000721  0.99998697 -0.07999287  0.99998573]
[-0.25000721  0.99998697 -0.07999287  0.99998573]
[-0.25000647  0.99998608 -0.07999385  0.9999843 ]
[-0.25000788  0.9999873  -0.07999612  0.99998474]
[-0.24999672  0.9999867  -0.07999837  0.99998753]
[-0.24999625  0.99998722 -0.07999789  0.99998813]
[-0.25000244  0.99998415 -0.08000452  0.99998585]
[-0.25000244  0.99998415 -0.08000452  0.99998585]
[-0.25000149  0.99996972 -0.08001488  0.99999186]
[-0.25000833  0.99995134 -0.08003405  0.99997716]
[-0.25000837  0.99995136 -0.08003405  0.99997719]
[-0.25000251  0.99997904 -0.08004632  0.99996331]
[-0.24997407  0.99998045 -0.08002886  0.99998352]
[-0.24997393  0.9999806  -0.08002889  0.99998355]
[-0.24997492  0.99998471 -0.08003261  0.99997907]
[-0.24997395  0.99998537 -0.08003208  0.99997987]
[-0.24997319  0.99998974 -0.08003237  0.99997876]
[-0.24997458  0.9999854  -0.08003822  0.9999731 ]
[-0.24997513  0.99998499